In [46]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset,TensorDataset,RandomSampler

import pandas as pd
import numpy as np
import re
import time

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


Data is available at [Link](https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews)

In [48]:
def decontract(sentence):
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

In [49]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

In [50]:
data = pd.read_csv("data/reviews.csv")
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [51]:
X = data[data['Text'].apply(lambda x: len(x.strip().split(" "))<50)]['Text']
print(len(X)) # 241812
max_len = max([len(x) for x in X])
y = data.loc[data[data['Text'].apply(lambda x: len(x.split(" "))<50)].index]['Score'].apply(lambda x: int(x>=3)).values
# y = F.one_hot(torch.tensor(y),num_classes=2).to(dtype=torch.float32)

241812


In [52]:
def normaliseText(X):
    X = X.str.lower()
    X = X.apply(decontract)
    X = X.apply(cleanPunc)
    X = X.apply(keepAlpha)
    X = X.apply(removeStopWords)
    X = X.apply(lambda x: re.sub(r'(\w)(\1{2,})', r'\1',x)) 
    return X
X = normaliseText(X)

In [53]:
class Words:
    def __init__(self):
        self.words2index={}
        self.index2words={}
        self.word_count = {}
        self.n_words = 1
    def addWord(self,word:str):
        if word not in self.word_count.keys():
            self.words2index[word] = self.n_words
            self.index2words[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            self.word_count[word]+=1

    def addSentence(self,sentence:str):
        for word in sentence.split(" "):
            self.addWord(word)
    
    def encode(self,sentence:str):
        encoded_rep = list()
        for word in sentence.split(" "):
            try:
                encoded_word = self.words2index[word.lower()]
            except KeyError as e:
                print(f"Adding {word} and encoding")
                self.addWord(word.lower())
                encoded_word = self.words2index[word.lower()]
            encoded_rep.append(encoded_word)
        return encoded_rep
    
    def decode(self,sentence:str):
        decoded_rep = list()
        for encoded_rep in sentence.split(" "):
            if encoded_rep == 100000:
                continue
            decoded_word = self.index2words[encoded_rep]
            decoded_rep.append(decoded_word)
        return decoded_rep
words = Words()

In [54]:
X.apply(words.addSentence)
print(f"Num of words: {words.n_words}")

Num of words: 54807


In [55]:
max_len = max(X.apply(lambda x:len(x.strip().split(" "))))
max_len

55

In [56]:
def pad_sequences(text:list):
    return text + [0] * (max_len-len(text))
def prepareData(data:list):
    input_tensor = torch.zeros((len(data),max_len),dtype=torch.long)
    for i,x in enumerate(data):
        x = words.encode(x)
        x = pad_sequences(x)
        input_tensor[i] = torch.tensor(x,dtype=torch.long)
    return input_tensor
input_tensor = prepareData(X.values)

In [57]:
X_train, X_val, y_train, y_val = train_test_split(input_tensor, y, 
                                                    test_size=0.2, random_state=1000)

In [58]:
TRAIN_BUFFER_SIZE = len(X_train)
VAL_BUFFER_SIZE = len(X_val)
BATCH_SIZE = 64

TRAIN_N_BATCH = TRAIN_BUFFER_SIZE // BATCH_SIZE
VAL_N_BATCH = VAL_BUFFER_SIZE // BATCH_SIZE

In [59]:
class Reviews(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [60]:
train_dataset = Reviews(X_train, y_train)
val_dataset = Reviews(X_val, y_val)

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE,
                          drop_last=True, shuffle=True)
val_dataset = DataLoader(val_dataset, batch_size = BATCH_SIZE,
                          drop_last=True, shuffle=True)

In [101]:
class Attention(nn.Module):
    def __init__(self,head_size, n_embed):
        super().__init__()
        self.keys = nn.Linear(n_embed,head_size,bias=False)
        self.queries = nn.Linear(n_embed,head_size,bias=False)
        self.values = nn.Linear(n_embed,head_size,bias=False)
    
    def __call__(self,X):
        # X = (64, 55, 500) 
        K = self.keys(X) # (64,55,500) @ (500,16) -> (64, 55, 16)
        Q = self.queries(X) # (64,55,500) @ (500,16) -> (64, 55, 16)
        V = self.values(X) # (64,55,500) @ (500,16) -> (64, 55, 16)

        e = Q @ K.transpose(1,2) / (K.shape[-1]**0.5) # (64, 55, 16) @ (64,16,55) -> (64,55,55) 
        weights = e.masked_fill((torch.tril(torch.ones(X.shape[1], X.shape[1]))==0).to(device),-torch.inf) # (64,55,55)
        weights = F.softmax(weights) 
        out = weights @ V # (64,55,55) @ (64,55,16) -> (64,55,16)
        return out
    

In [102]:
embedding_dim = 256
units = 500
target_size = 2
vocab_size = words.n_words
PATH = './model/model.pt'

In [92]:
class Model(nn.Module):
    def __init__(self,embedding_dim,vocab_size,hidden_size,target_size,head_size):
        super().__init__()
        self.embed = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim,padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,hidden_size,batch_first=True)
        self.attention = Attention(head_size=head_size,n_embed=hidden_size).to(device)
        self.batchnorm = nn.BatchNorm1d(max_len*head_size)
        self.tanh = nn.Tanh()
        self.linear = nn.Linear(max_len*head_size,target_size,bias=True)
        self.batch_size = BATCH_SIZE
        self.hidden_size = hidden_size
    
    def init_hidden(self):
        return (torch.ones((1,self.batch_size,self.hidden_size),device=device),torch.ones((1,self.batch_size,self.hidden_size),device=device))
    def forward(self,X):
        # X = (64,55)
        x = self.embed(X) # (64,55) -> (64,55,256)
        (h_n,c_n) = self.init_hidden()
        x,(hidden,cell) = self.lstm(x,(h_n,c_n)) # (64,55,256) -> (64,55,500)
        output = self.attention(x) # (64,55,500) -> (64,55,16)
        output = output.view(output.size(0),-1) # (64,55,16) -> (64,880) # [64,55*16] 
        output = self.batchnorm(output) # (64,800) -> (64,800)
        output = self.tanh(output)
        output = self.linear(output) # (64,880) -> (64,2)

        return output

In [103]:
model = Model(embedding_dim,vocab_size,units,target_size,16).to(device=device)
print(model)

Model(
  (embed): Embedding(54807, 256, padding_idx=0)
  (lstm): LSTM(256, 500, batch_first=True)
  (attention): Attention(
    (keys): Linear(in_features=500, out_features=16, bias=False)
    (queries): Linear(in_features=500, out_features=16, bias=False)
    (values): Linear(in_features=500, out_features=16, bias=False)
  )
  (batchnorm): BatchNorm1d(880, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh): Tanh()
  (linear): Linear(in_features=880, out_features=2, bias=True)
)


In [104]:
def accuracy(targets, predictions):
    correct = (targets.argmax(1).to(device) == predictions.argmax(1).to(device)).sum()
    accuracy = 100. * correct / len(predictions)
    return accuracy

def save_model(EPOCH,model,optim,LOSS,PATH):
    torch.save({
            'epoch': EPOCH,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': LOSS,
            }, PATH)
def training(train_data,val_data,criterion,optim,epochs,model,batch_interval=100,epoch_interval=100):
    start = time.time()
    for epoch in range(epochs):
        model.train()
        print("training")
        train_total_accuracy, val_total_accuracy = 0,0
        train_total_count, val_total_count = 0,0
        for (batch,(X,y)) in enumerate(train_data):
            y = y.to(device)
            logits = model(X.to(device))
            loss=criterion(logits,y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            train_total_accuracy += (logits.argmax(1) == y).sum().item()
            train_total_count += y.size(0)
            if batch % batch_interval == 0:
                save_model(epoch,model,optim,loss,PATH)
                print(f"{epoch:3d}: [{batch:4d}/{len(train_data):4d}] {(train_total_accuracy/train_total_count):.4f}")
        model.eval()
        for (batch,(X,y)) in enumerate(val_data):
            y = y.to(device)
            logits = model(X.to(device))
            val_total_accuracy += (logits.argmax(1) == y).sum().item()
            val_total_count += y.size(0)
            if batch % batch_interval == 0:
                print(f"{epoch:3d}: [{batch:4d}/{len(val_data):4d}] {(val_total_accuracy/val_total_count):.4f}")
        print(f"{batch}/{len(val_data)} validation")
        if epoch % epoch_interval == 0:
            print(f"epoch:{epoch}/{epochs} train_accuracy:{train_total_accuracy/train_total_count} | val_accuracy:{val_total_accuracy/val_total_count}")
    print(f"{time.time()-start} secs")

In [105]:
epochs = 5
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(),lr=1e-4)
training(train_dataset,val_dataset,criterion,optim,epochs,model,epoch_interval=1)

training


/var/folders/b4/9cg_tkrn3256x3hcybpxqnpw0000gn/T/ipykernel_15007/257930496.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(weights)


  0: [   0/3022] 0.3906
  0: [ 100/3022] 0.8696
  0: [ 200/3022] 0.8755
  0: [ 300/3022] 0.8770
  0: [ 400/3022] 0.8771
  0: [ 500/3022] 0.8768
  0: [ 600/3022] 0.8768
  0: [ 700/3022] 0.8766
  0: [ 800/3022] 0.8775
  0: [ 900/3022] 0.8774
  0: [1000/3022] 0.8774
  0: [1100/3022] 0.8769
  0: [1200/3022] 0.8770
  0: [1300/3022] 0.8772
  0: [1400/3022] 0.8775
  0: [1500/3022] 0.8779
  0: [1600/3022] 0.8776
  0: [1700/3022] 0.8776
  0: [1800/3022] 0.8778
  0: [1900/3022] 0.8778
  0: [2000/3022] 0.8778
  0: [2100/3022] 0.8775
  0: [2200/3022] 0.8776
  0: [2300/3022] 0.8776
  0: [2400/3022] 0.8776
  0: [2500/3022] 0.8774
  0: [2600/3022] 0.8778
  0: [2700/3022] 0.8779
  0: [2800/3022] 0.8780
  0: [2900/3022] 0.8779
  0: [3000/3022] 0.8780
  0: [   0/ 755] 0.8906
  0: [ 100/ 755] 0.8820
  0: [ 200/ 755] 0.8827
  0: [ 300/ 755] 0.8825
  0: [ 400/ 755] 0.8825
  0: [ 500/ 755] 0.8806
  0: [ 600/ 755] 0.8790
  0: [ 700/ 755] 0.8805
754/755 validation
epoch:0/5 train_accuracy:0.8779781601588352 |

In [106]:
torch.save(model.state_dict(),"./model/senti-attention.pt")